# Microstructure characterization of battery materials based on voxelated image data: Computation of active surface area and tortuosity

## Battery microstructures

The study is complemented by three realistic microstructures, namely a
representative volume element of nano-porous NVP-C [Neumann et. al. 2024](https://chemistry-europe.onlinelibrary.wiley.com/doi/10.1002/batt.202300409) which is a promising cathode
material for sodium ion batteries, a section from a commercial NMC
electrode from x-ray tomography [Usseglio-Viretta et. al. 2018](https://iopscience.iop.org/article/10.1149/2.0731814jes/pdf) and a biphasic iron-graphite structure from [microlib](https://microlib.io) to underline the generality of the used methods.
<br>


<p style="display: inline-block; margin-right: 10px;">
  <img src="data-paper/battery-structures/nanoporous-NVP.png" alt="Image 1" width="300" />
</p>
<p style="display: inline-block; margin-right: 10px;">
  <img src="data-paper/battery-structures/xray-electrode-CBD.png" alt="Image 1" width="300" />
</p>
<p style="display: inline-block; margin-right: 10px;">
  <img src="data-paper/battery-structures/graphite-ferrite.png" alt="Image 1" width="300" />
</p>


**Author**: Simon Daubner<br>
**Department**: Department of Mechanical Engineering<br>
**Institution**: Karlsruhe Institute of Technology

In [3]:
import numpy as np
import tifffile

import metrics
import data

### NVP-C: nanoporous cathode material for sodium ion batteries

In [4]:
path_to_images = "/.../VirtualStructures/Model_NVP_60/Model_NVP_60_VolC_150_N=01"
# path_to_images = "/.../VirtualStructures/Model_NVP_60/Model_NVP_60_VolC_275_N=01"
# path_to_images = "/.../VirtualStructures/Model_NVP_60/Model_NVP_60_VolC_300_N=01"
# structure = data.read_image_stack_pgm(path_to_images)
# print("Stack shape:", structure.shape)

# NVP = 255, carbon = 0, pore = 155
# labels = {"NVP":255, "carbon":0, "pore":155}
# px = 16.6e-9 # pixel resolution in m

### Full electrode example: NMC commercial electrode

In [5]:
path_to_tif = "data-paper/battery-structures/nmc-1-cal-withcbd-w099-binarized.tif"
tif_file = tifffile.imread(path_to_tif)
structure = np.array(tif_file)
structure = structure[20:-20,20:-20,20:-20]
print("Stack shape:", structure.shape)

labels = {"pore":0, "NMC":1, "CBD":2}
px = 398e-9 # pixel resolution in m

Stack shape: (213, 213, 213)


### Biphasic iron-graphite structure from [microlib](https://microlib.io)

**Description:** The addition of a substantial amount of silicon to a relatively low carbon cast iron serves to induce the cementite to transform to ferrite and graphite flakes. This results in the properties of the resultant metal being more similar to that of grey cast iron, i.e. making it easier to machine and improving wear resistance.<br>
**Contributor:** Dr R F Cochrane<br>
**Organisation:** Department of Materials, University of Leeds<br>
**Voxel size:** 1.375 µm

In [4]:
#path_to_tif = "/.../microstructure378.tif"
#tif_file = tifffile.imread(path_to_tif)
#structure = np.array(tif_file)
#print("Stack shape:", structure.shape)

#labels = {"ferrite":1, "graphite":0}
#px = 1.375e-6 # pixel resolution in m

### Generic script for microstructure metrics

Load one of the three samples above or try your own data. The following code snippets will extract the

- volume fraction,
- surface area,
- pairwise-surface area,
- through-feature connectivities and
- tortuosity factors

of all phases defined by the `labels` dictionary.

#### Volume fractions

In [6]:
volume_fraction = {}

for key, value in labels.items():
    # Compute volume fraction of full datset
    volume_fraction[key] = metrics.volume_fraction(structure , value)
    print(f"Volume fraction of {key}: {volume_fraction[key]:.4f}")

Volume fraction of pore: 0.3598
Volume fraction of NMC: 0.5062
Volume fraction of CBD: 0.1340


#### Surface areas

In [6]:
surface = {}
for key, value in labels.items():
    surface[key] = metrics.specific_surface_area((structure == value).astype(float), dx=px, dy=px, dz=px)
    print(f"Specific surface area of {key}: {surface[key]*1e-6:.5f} [1/µm]")

for key1, value in labels.items():
    for key2, value in labels.items():
        if key1 != key2:
            for key3, value in labels.items():
                if key3 != key2 and key3 != key1:
                    area12 = 0.5*(surface[key1] + surface[key2] - surface[key3])
                    area13 = 0.5*(surface[key1] + surface[key3] - surface[key2])
                    area23 = 0.5*(surface[key2] + surface[key3] - surface[key1])
                    print(f"Specific surface area of {key1}-{key2}:  {area12*1e-6:.5f} [1/µm]")
                    print(f"Specific surface area of {key1}-{key3}:  {area13*1e-6:.5f} [1/µm]")
                    print(f"Specific surface area of {key2}-{key3}:  {area23*1e-6:.5f} [1/µm]")
            break
    break

Specific surface area of pore: 0.43715 [1/µm]
Specific surface area of NMC: 0.27315 [1/µm]
Specific surface area of CBD: 0.46772 [1/µm]
Specific surface area of pore-NMC:  0.12129 [1/µm]
Specific surface area of pore-CBD:  0.31586 [1/µm]
Specific surface area of NMC-CBD:  0.15186 [1/µm]


#### Through feature connectivity

In [7]:
through_fraction = {}
for key, value in labels.items():
    through_fraction[key] = {}
    _, through_fraction[key]['x'] = metrics.extract_through_feature(structure, value, 'x')
    _, through_fraction[key]['y'] = metrics.extract_through_feature(structure, value, 'y')
    _, through_fraction[key]['z'] = metrics.extract_through_feature(structure, value, 'z')

print( "With |  side  |  edge  | corner | connectivity")
for key, value in labels.items():
    print(f" x = | {through_fraction[key]['x'][0]:.4f} | {through_fraction[key]['x'][1]:.4f} | {through_fraction[key]['x'][2]:.4f} | % of {key} are connected")
    print(f" y = | {through_fraction[key]['y'][0]:.4f} | {through_fraction[key]['y'][1]:.4f} | {through_fraction[key]['y'][2]:.4f} | % of {key} are connected")
    print(f" z = | {through_fraction[key]['z'][0]:.4f} | {through_fraction[key]['z'][1]:.4f} | {through_fraction[key]['z'][2]:.4f} | % of {key} are connected")

With |  side  |  edge  | corner | connectivity
 x = | 0.9898 | 0.9978 | 0.9987 | % of pore are connected
 y = | 0.9898 | 0.9978 | 0.9987 | % of pore are connected
 z = | 0.9898 | 0.9978 | 0.9987 | % of pore are connected
 x = | 0.9974 | 0.9985 | 0.9985 | % of NMC are connected
 y = | 0.9974 | 0.9985 | 0.9985 | % of NMC are connected
 z = | 0.9974 | 0.9985 | 0.9985 | % of NMC are connected
 x = | 0.9772 | 0.9940 | 0.9963 | % of CBD are connected
 y = | 0.9772 | 0.9940 | 0.9963 | % of CBD are connected
 z = | 0.9772 | 0.9940 | 0.9963 | % of CBD are connected


#### Tortuosity factors

In [8]:
tau = {}
for key, value in labels.items():
    tau_vgl = metrics.tortuosity(structure == value)
    through_feature, _ = metrics.extract_through_feature(structure, value, 'x', connectivity=1)
    tau[key] = metrics.tortuosity(through_feature[0])
    print(f"Tortuosity factor of {key} in x: {tau[key]:.5f} ({tau_vgl:.5f}).")
    tau_vgl = metrics.tortuosity(np.transpose(structure == value, (1,0,2)))
    through_feature, _ = metrics.extract_through_feature(structure, value, 'y', connectivity=1)
    tau[key] = metrics.tortuosity(np.transpose(through_feature[0], (1,0,2)))
    print(f"Tortuosity factor of {key} in y: {tau[key]:.5f} ({tau_vgl:.5f}).")
    tau_vgl = metrics.tortuosity(np.transpose(structure == value, (2,0,1)))
    through_feature, _ = metrics.extract_through_feature(structure, value, 'z', connectivity=1)
    tau[key] = metrics.tortuosity(np.transpose(through_feature[0], (2,0,1)))
    print(f"Tortuosity factor of {key} in z: {tau[key]:.5f} ({tau_vgl:.5f}).")

converged to: 2.7050857543945312                   after: 701 iterations in: 36.3643                    seconds at a rate of 0.0519 s/iter
converged to: 2.6776435375213623                   after: 701 iterations in: 36.2832                    seconds at a rate of 0.0518 s/iter
Tortuosity factor of pore in x: 2.67764 (2.70509).
converged to: 2.798722505569458                   after: 801 iterations in: 41.3753                    seconds at a rate of 0.0517 s/iter
converged to: 2.770310878753662                   after: 801 iterations in: 41.3653                    seconds at a rate of 0.0516 s/iter
Tortuosity factor of pore in y: 2.77031 (2.79872).
converged to: 3.294743061065674                   after: 801 iterations in: 41.4986                    seconds at a rate of 0.0518 s/iter
converged to: 3.2612972259521484                   after: 801 iterations in: 41.7399                    seconds at a rate of 0.0521 s/iter
Tortuosity factor of pore in z: 3.26130 (3.29474).
converged to: 2.